In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 18kB/s 


In [ ]:
import tensorflow as tf

In [ ]:
def max_pool_with_argmax(net,stride):

    #使用mask保存每个最大值的位置 这个函数只支持GPU操作
    _, mask = tf.nn.max_pool_with_argmax( net,ksize=[1, stride, stride, 1], strides=[1, stride, stride, 1],padding='SAME')
    #将反向传播的mask梯度计算停止
    mask = tf.stop_gradient(mask)
    #计算最大池化操作
    net = tf.nn.max_pool(net, ksize=[1, stride, stride, 1],strides=[1, stride, stride, 1], padding='SAME')
    #将池化结果和mask返回
    return net,mask
 
 
 
def un_max_pool(net,mask,stride):

    ksize = [1, stride, stride, 1]
    input_shape = net.get_shape().as_list()
    # calculation new shape
    output_shape = (input_shape[0], input_shape[1] * ksize[1], input_shape[2] * ksize[2], input_shape[3])
    # calculation indices for batch, height, width and feature maps
    one_like_mask = tf.ones_like(mask)
    batch_range = tf.reshape(tf.range(output_shape[0], dtype=tf.int64), shape=[input_shape[0], 1, 1, 1])
    b = one_like_mask * batch_range
    y = mask // (output_shape[2] * output_shape[3])
    x = mask % (output_shape[2] * output_shape[3]) // output_shape[3]
    feature_range = tf.range(output_shape[3], dtype=tf.int64)
    f = one_like_mask * feature_range
    # transpose indices & reshape update values to one dimension
    updates_size = tf.size(net)
    indices = tf.transpose(tf.reshape(tf.stack([b, y, x, f]), [4, updates_size]))
    values = tf.reshape(net, [updates_size])
    ret = tf.scatter_nd(indices, values, output_shape)
    return ret
 


In [ ]:
#定义一个形状为4x4x2的张量
img = tf.constant([
[[0.0,4.0],[0.0,4.0],[0.0,4.0],[0.0,4.0]],
[[1.0,5.0],[1.0,5.0],[1.0,5.0],[1.0,5.0]],
[[2.0,6.0],[2.0,8.0],[2.0,6.0],[2.0,6.0]],
[[3.0,7.0],[3.0,7.0],[3.0,7.0],[3.0,7.0]],
])
img = tf.reshape(img,[1,4,4,2])
#最大池化操作
pooling1 = tf.nn.max_pool(img,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
#带有最大值位置的最大池化操作
pooling2,mask = max_pool_with_argmax(img,2)
#反最大池化
img2 = un_max_pool(pooling2,mask,2)

In [ ]:
pooling2

<tf.Tensor: shape=(1, 2, 2, 2), dtype=float32, numpy=
array([[[[1., 5.],
         [1., 5.]],

        [[3., 8.],
         [3., 7.]]]], dtype=float32)>

In [ ]:
mask

<tf.Tensor: shape=(1, 2, 2, 2), dtype=int64, numpy=
array([[[[ 8,  9],
         [12, 13]],

        [[24, 19],
         [28, 29]]]])>

In [ ]:
img2

<tf.Tensor: shape=(1, 4, 4, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[1., 5.],
         [0., 0.],
         [1., 5.],
         [0., 0.]],

        [[0., 0.],
         [0., 8.],
         [0., 0.],
         [0., 0.]],

        [[3., 0.],
         [0., 0.],
         [3., 7.],
         [0., 0.]]]], dtype=float32)>